In [1]:
from tkinter import Tk, Canvas
import random
import numpy as np

In [2]:
def get_mazetrix():
    with open("mazetrix.txt") as f:
        lines = f.read().splitlines()
        return [line.strip() for line in lines]

In [3]:
def get_mouse_xy():
    for y, row in enumerate(mazetrix):
        #print(y,' ',row)
        x = row.find("S")
        #print(x,' ',y)
        if(x != -1):
            return x, y

In [4]:
def draw_maze():
    for y, row in enumerate(mazetrix):
        for x, ch in enumerate(row):
            if(is_wall(x, y)):
                draw_wall(x, y)

In [5]:
def draw_wall(x, y):
    draw(x, y, "black")

In [6]:
def draw_mouse(x, y):
    draw(x, y, "blue")

In [7]:
def draw_footprint(x, y):
    draw(x, y, "yellow")

In [8]:
def draw(x, y, color):

    maze_height = len(mazetrix)
    maze_width = len(mazetrix[0])
    #print(maze_height,' ',maze_width)
    height = canvas_height/maze_height
    width = canvas_width/maze_width

    x1, y1 = (x*width, y*height)
    x2, y2 = (x1+width, y1+height)
    w.create_rectangle(x1, y1, x2, y2, fill=color, outline=color)

In [9]:
def is_wall(x, y):
    return mazetrix[y][x] == '#'

In [10]:
def is_exit(x, y):
    return mazetrix[y][x] == 'E'

In [11]:
canvas_height = 450
canvas_width = 750

In [12]:
def is_terminal_state(current_row_index, current_column_index):
    if rewards[current_row_index, current_column_index] == -1.:
        return False
    else:
        return True

def get_starting_location():
    maze_height = len(mazetrix)
    maze_width = len(mazetrix[0])
    current_row_index = np.random.randint(maze_height)
    current_column_index = np.random.randint(maze_width)
    while is_terminal_state(current_row_index, current_column_index):
        current_row_index = np.random.randint(maze_height)
        current_column_index = np.random.randint(maze_width)
    return current_row_index, current_column_index

def get_next_action(current_row_index, current_column_index, epsilon):
    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row_index, current_column_index])
    else: #choose a random action
        return np.random.randint(4)

def get_next_location(current_row_index, current_column_index, action_index):
    maze_height = len(mazetrix)
    maze_width = len(mazetrix[0])
    new_row_index = current_row_index
    new_column_index = current_column_index
    if actions[action_index] == 'up' and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 'right' and current_column_index < maze_width - 1:
        new_column_index += 1
    elif actions[action_index] == 'down' and current_row_index < maze_height - 1:
        new_row_index += 1
    elif actions[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1
    return new_row_index, new_column_index


In [13]:
def q_learning_algorithm():
    maze_height = len(mazetrix)
    maze_width = len(mazetrix[0])
    global rewards,q_values,actions
    
    q_values = np.zeros((maze_height, maze_width, 4))
    
    actions = ['up', 'right', 'down', 'left']
    
    
    rewards = np.full((maze_height, maze_width), -1.)
    for y, row in enumerate(mazetrix):
        for x, ch in enumerate(row):
            if(mazetrix[y][x]=='#'):
                rewards[y][x]=-100
            elif(mazetrix[y][x]=='E'):
                rewards[y][x]=100
    #for row in reward_val:
        #print(row)
        
    epsilon = 0.9          
    discount_factor = 0.9  
    learning_rate = 0.9   

    for episode in range(10000):
        row_index, column_index = get_starting_location()

        while not is_terminal_state(row_index, column_index):
            action_index = get_next_action(row_index, column_index, epsilon)

            old_row_index, old_column_index = row_index, column_index #store the old row and column indexes
            row_index, column_index = get_next_location(row_index, column_index, action_index)

            reward = rewards[row_index, column_index]
            old_q_value = q_values[old_row_index, old_column_index, action_index]
            temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

            new_q_value = old_q_value + (learning_rate * temporal_difference)
            q_values[old_row_index, old_column_index, action_index] = new_q_value

    print('Training complete!')
    

In [14]:
def q_learning_train():
    global mazetrix
    mazetrix = get_mazetrix()
    q_learning_algorithm()

In [15]:
q_learning_train()

Training complete!


In [16]:
def move_the_mouse(x,y):
    new_x=x
    x=y
    y=new_x
    if is_terminal_state(x, y):
        return []

    while not is_terminal_state(x, y):
        draw_footprint(y, x)
        action_index = get_next_action(x, y, 1.)
        x, y = get_next_location(x, y, action_index)
        draw_mouse(y, x)
        w.pack()
        if(is_exit(y, x)):
            break

In [20]:
def main():

    global root, w, mazetrix, x, y
    mazetrix = get_mazetrix()
    x, y = get_mouse_xy()

    root = Tk()
    root.title("Path Finding")
    w = Canvas(root, bg="white", width=canvas_width, height=canvas_height)
    w.pack()
    draw_maze()
    move_the_mouse(x,y)
    root.mainloop()

In [21]:
main()